In [1]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

In [2]:
from __future__ import print_function
import torch
from torch.utils.data import DataLoader, random_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]

Gs,y = loadDataset('/home/ines/Documents/M2/Stage/stage_ged/Ines/DeepGED/MAO/dataset.ds')
#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[1]
G2 = Gs[9]
print(y[1],y[9])

'''
plt.figure(0)
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')

plt.figure(1)
nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')

plt.show()
'''

import extended_label
for g in Gs:
    extended_label.compute_extended_labels(g)
#for v in Gs[10].nodes():
#        print(Gs[10].nodes[v])

#print(nx.to_dict_of_lists(Gs[13]))



#dict={'C':0,'N':1,'O':2}
#A,labels=from_networkx_to_tensor2(Gs[13],dict)
#print(A)
#A1=(A==torch.ones(13,13)).int()
#A2=(A==2*torch.ones(13,13)).int()
#print(A1)
#print(A2)


edge max label 3
0.0 0.0


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import svd
import rings
from svd import iterated_power as compute_major_axis

#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False,node_label='label'):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.node_label=node_label
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print(self.nb_edge_labels)
        self.device='cuda' if torch.cuda.is_available() else 'cpu'
        nb_node_pair_label=self.nb_labels*(self.nb_labels-1)/2.0
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        
        self.node_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(int(self.nb_labels*(self.nb_labels-1)/2+1),requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del        
        self.edge_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
        
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.empty((len(GraphList),card_max*card_max),dtype=torch.int,device=self.device)
        self.labels=torch.empty((len(GraphList),card_max),dtype=torch.int,device=self.device)
        print(self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,0:l.shape[0]]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        
    def forward(self, input):        
        ged=torch.zeros(len(input)).to(self.device) 
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
        
            
        
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            n=self.card[g1]
            m=self.card[g2]
            
            self.ring_g,self.ring_h = rings.build_rings(g1,edgeInsDel.size()), rings.build_rings(g2,edgeInsDel.size()) 
            
            C=self.construct_cost_matrix(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)      
            #S=self.mapping_from_similarity(C,n,m)
            #S=self.mapping_from_cost(C,n,m)   
            #S=self.new_mapping_from_cost(C,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
            S=self.mapping_from_cost_sans_FW(n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
            
            v=torch.flatten(S)
            
            normalize_factor=1.0
            if self.normalize:
                nb_edge1=(self.A[g1][0:n*n] != torch.zeros(n*n,device=self.device)).int().sum()
                nb_edge2=(self.A[g2][0:m*m] != torch.zeros(m*m,device=self.device)).int().sum()
                normalize_factor=nodeInsDel*(n+m)+edgeInsDel*(nb_edge1+nb_edge2)
            c=torch.diag(C)
            D=C-torch.eye(C.shape[0],device=self.device)*c
            ged[k]=(.5*v.T@D@v+c.T@v)/normalize_factor
        max=torch.max(ged)
        min=torch.min(ged)
        ged=(ged-min)/(max-min)
        
        return ged
    
    def from_weighs_to_costs(self):
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        ce=self.edge_weighs*self.edge_weighs
        total_cost=cn.sum()+ce.sum()
        cn=cn/total_cost
        ce=ce/total_cost
        edgeInsDel=ce[-1]

        node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        node_costs=node_costs+node_costs.T

        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        return node_costs,cn[-1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v][self.node_label][0] in node_labels:
                    node_labels.append(G.nodes[v][self.node_label][0])
        node_labels.sort()
        #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print(node_labels)
        print(dict,len(dict))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict):    
        A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=int,weight='bond_type').todense(),dtype=torch.int)        
        lab=[dict[G.nodes[v][self.node_label][0]] for v in nx.nodes(G)]
   
        return (A.view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))

    def construct_cost_matrix(self,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
        A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
        A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
        A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)
        
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
#                    C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1),alpha=edge_costs[k][l])
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1)
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

        l1=self.labels[g1][0:n]
        l2=self.labels[g2][0:m]
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([node_costs[l1[k//(m+1)],l2[k%(m+1)]] for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
      
        return C
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
    
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1)
    
    def similarity_from_cost(self,C):
        N=C.shape[0]
             
        #return (torch.norm(C,p='fro')*torch.eye(N,device=self.device) -C)
        return (C.max()*torch.eye(N,device=self.device) -C)
    
    def lsape_populate_instance(self,first_graph,second_graph,average_node_cost, average_edge_cost,alpha,lbda):       #ring_g, ring_h come from global ring with all graphs in so ring_g = rings['g'] and ring_h = rings['h']
        g,h = Gs[first_graph], Gs[second_graph]
        self.average_cost =[average_node_cost, average_edge_cost]
        self.first_graph, self.second_graph = first_graph,second_graph
        
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()

        lsape_instance = [[0 for _ in range(len(g) + 1)] for __ in range(len(h) + 1)]
        for g_node_index in range(len(g) + 1):
            for h_node_index in range(len(h) + 1):
                lsape_instance[h_node_index][g_node_index] = rings.compute_ring_distance(g,h,self.ring_g,self.ring_h,g_node_index,h_node_index,alpha,lbda,node_costs,nodeInsDel,edge_costs,edgeInsDel,first_graph,second_graph)
        for i in lsape_instance :
            i = torch.as_tensor(i)
        lsape_instance = torch.as_tensor(lsape_instance)
        #print(type(lsape_instance))
        return lsape_instance
    
  
    def mapping_from_cost_sans_FW(self,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel): 
        c_0 =self.lsape_populate_instance(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
        x0=svd.eps_assigment_from_mapping(torch.exp(-c_0),10).view((n+1)*(m+1),1)
        return x0
    
    def new_mapping_from_cost(self,C,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel): 
        c=torch.diag(C)       
        c_0 =self.lsape_populate_instance(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-c_0),10).view((n+1)*(m+1),1)
        return svd.franck_wolfe(x0,D,c,5,15,n,m)
    
    
    def mapping_from_cost(self,C,n,m):
        c=torch.diag(C)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-.5*c.view(n+1,m+1)),10).view((n+1)*(m+1),1)
        x=svd.franck_wolfe(x0,D,c,5,10,n,m)
        def print_grad(grad):
            if(grad.norm()!= 0.0):
                print(grad)
        
#        x.register_hook(print_grad)
        return x

print('Gs=',len(Gs))
model = Net(Gs,normalize=True,node_label='extended_label')

params = list(model.parameters())
print(len(params))
print(params[0])
#print(model(input))
print(max([G.order() for G in Gs]),len(Gs))
print('toto')

Gs= 68
['C_1C', 'C_1C1C1N', 'C_1C1C2C', 'C_1C1N', 'C_1C1N2C', 'C_1C1O', 'C_1C1O2C', 'C_1C2C', 'C_1C3C', 'C_1N', 'C_1O', 'C_2C', 'C_2C2C', 'C_3C', 'N_1C', 'N_1C1C', 'N_1C1C1C', 'O_1C', 'O_1C1C']
{'C_1C': 0, 'C_1C1C1N': 1, 'C_1C1C2C': 2, 'C_1C1N': 3, 'C_1C1N2C': 4, 'C_1C1O': 5, 'C_1C1O2C': 6, 'C_1C2C': 7, 'C_1C3C': 8, 'C_1N': 9, 'C_1O': 10, 'C_2C': 11, 'C_2C2C': 12, 'C_3C': 13, 'N_1C': 14, 'N_1C1C': 15, 'N_1C1C1C': 16, 'O_1C': 17, 'O_1C1C': 18} 19
3
torch.Size([68, 729])
adjacency matrices tensor([[         0,          1,          0,  ..., 1634951028, 1180660590,
          745416791],
        [         0,          1,          0,  ..., 1697411956, 1600481124,
         1953722211],
        [         0,          1,          0,  ..., 1735289202, 1953068832,
         1818304616],
        ...,
        [         0,          1,          0,  ...,  778593138,  745827945,
         1769366884],
        [         0,          1,          0,  ..., 1685024351, 1868783461,
          539784307],
        [

In [5]:
import itertools

nb=len(Gs)
class1=torch.tensor([k for k in range(len(y)) if y[k]==1])
class2=torch.tensor([k for k in range(len(y)) if y[k]==0])

nb_class1=12
nb_class2=int((nb_class1-1)/2)
train_size=nb_class1+nb_class2
#train_size=20

#if train_size % 2 == 0:
#    nb_class1=int(train_size/2)
#    nb_class2=int(train_size/2)
#else:
#    nb_class1=int(train_size/2)+1
#    nb_class2=int(train_size/2)
    
print((torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long())
random_class1=class1[(torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long()]
random_class2=class2[(torch.abs(10000*torch.randn(nb_class2)).int()%class2.size()[0]).long()]
train_graphs=torch.cat((random_class1,random_class2),0)
print('train graphs:',train_graphs)


couples=torch.triu_indices(train_size,train_size,offset=1)
print('couples=',couples)
print('nb_class1/nb_class2=',nb_class1,nb_class2)
#combinations=itertools.combinations(range(nb),2)

nb_elt=int(nb_class1*(nb_class1+2*nb_class2-1)/2)
print('couples restreints:',couples[:,0:nb_elt])

#nb_elt=int(train_size*(train_size-1)/2)
data=torch.empty((nb_elt,2),dtype=torch.int)
yt=torch.ones(nb_elt)
print('old_size, new size=',nb_elt,.5*nb_class1*(nb_class1+2*nb_class2-1))
data[0:nb_elt,0]=train_graphs[couples[0,0:nb_elt]]
data[0:nb_elt,1]=train_graphs[couples[1,0:nb_elt]]


#data[0:nb_elt,0]=train_graphs[couples[0]]
#data[0:nb_elt,1]=train_graphs[couples[1]]
print(nb_elt)
#couples=[]
for k in range(nb_elt):
    if (y[data[k][0]]!=y[data[k][1]]):
        yt[k]=-1.0        

print('data=',data)

#print(couples[1:2])

torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda:0")          # a CUDA device object    


tensor([19,  0, 22, 24,  2, 11,  8,  3, 11, 11, 27,  1])
train graphs: tensor([19, 65, 26, 45, 66,  7, 18, 38, 14, 19, 19, 60,  5, 62, 52, 52, 29])
couples= tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,
          5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,
          6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,
          9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11,
         12, 12, 12, 12, 13, 13, 13, 14, 14, 15],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  2,  3,
          4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  3,  4,  5,  6,  7,
          8,  9, 10, 11, 12, 13, 14, 15, 16,  4,  5,  6,  7,  8,

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from torch.utils.data import DataLoader, random_split, TensorDataset

[train_D, valid_D,train_L,valid_L]= train_test_split(data,yt, test_size=0.25,train_size=0.75, shuffle=True) #, stratify=yt)
#print(train_D)   
    
DatasetTrain = TensorDataset(train_D, train_L)

DatasetValid=TensorDataset(valid_D, valid_L)

trainloader=torch.utils.data.DataLoader(DatasetTrain,batch_size=16,shuffle=True,drop_last=True, num_workers=0)
#len(train_D) #31
validationloader=torch.utils.data.DataLoader(DatasetValid, batch_size=8, drop_last=True,num_workers=0)

print(len(train_D), len(valid_D))
print(len(trainloader),len(validationloader))
for i,j in validationloader:
    print('i : ',i,'\n j : ',j)

94 32
5 4
i :  tensor([[26, 45],
        [45, 19],
        [19, 52],
        [38, 52],
        [ 7, 29],
        [19, 18],
        [ 7, 52],
        [65, 62]], dtype=torch.int32) 
 j :  tensor([ 1.,  1., -1., -1., -1.,  1., -1., -1.])
i :  tensor([[45,  7],
        [45, 18],
        [65, 66],
        [66, 52],
        [65, 60],
        [66, 52],
        [45, 66],
        [19, 52]], dtype=torch.int32) 
 j :  tensor([ 1.,  1.,  1., -1.,  1., -1.,  1., -1.])
i :  tensor([[ 7, 52],
        [19, 19],
        [19, 26],
        [ 7, 38],
        [ 7, 60],
        [ 7, 19],
        [65, 26],
        [18, 38]], dtype=torch.int32) 
 j :  tensor([-1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])
i :  tensor([[26, 52],
        [45, 29],
        [18, 29],
        [14, 29],
        [60, 62],
        [65, 29],
        [65,  5],
        [18,  5]], dtype=torch.int32) 
 j :  tensor([-1., -1., -1., -1., -1., -1., -1., -1.])


In [7]:
#if torch.cuda.device_count() > 1:
#  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#  model = nn.DataParallel(model)
from triangular_losses import TriangularConstraint as triangular_constraint

model.to(device)
def classification(model,data,yt,nb_iter):

    criterion = torch.nn.HingeEmbeddingLoss(margin=1.0,reduction='mean')
    criterionTri=triangular_constraint()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    #print(device)

    #torch.cat((same_class[0:20],diff_class[0:20]),0).to(device)
    whole_input=data.to(device)
    target=yt.to(device) 
    #torch.ones(40,device=device)
    #target[20:]=-1.0
    #target=(yt[0:20]).to(device)
    InsDel=np.empty((nb_iter,2))
    node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
    nodeSub=np.empty((nb_iter,int(node_costs.shape[0]*(node_costs.shape[0]-1)/2)))
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    loss_train_plt=np.empty(nb_iter)
    loss_valid_plt=np.empty(nb_iter)
    min_valid_loss = np.inf
    iter_min_valid_loss = 0
    
    for t in range(nb_iter):    
        train_loss = 0.0
        valid_loss = 0.0
        tmp=np.inf
        for train_data,train_labels in trainloader:
            optimizer.zero_grad()
            
            inputt=train_data.to(device)
            
            # Forward pass: Compute predicted y by passing data to the model
            y_pred = model(whole_input).to(device)

            # Compute and print loss
            loss = criterion(y_pred, target).to(device)
            node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
            triangularInq=criterionTri(node_costs,nodeInsDel,edge_costs,edgeInsDel)
            loss=loss*(1+triangularInq)
            loss.to(device)
            # Zero gradients, perform a backward pass, and update the weights.
            #optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            print('loss.item of the train = ', t, loss.item())
            train_loss =+ loss.item() #* train_data.size(0) 
            if (loss.item()<tmp): tmp=loss.item()
        loss_plt[t]=loss.item()  
        loss_train_plt[t]=train_loss /len(trainloader)
        #loss_plt[t]=tmp
            
        if t % 100 == 99 or t==0:   
            print('ged=',y_pred*target)  #train_labels
            print('Distances: ',y_pred)
            print('Loss Triangular:',triangularInq.item())
            print('node_costs :')
            print(node_costs)
            print('nodeInsDel:',nodeInsDel.item())
            print('edge_costs :')
            print(edge_costs)
            print('edgeInsDel:',edgeInsDel.item())
            
            
            
        for valid_data,valid_labels in validationloader:
            
            inputt=valid_data.to(device)
            y_pred = model(inputt).to(device)
            # Compute and print loss
            loss = criterion(y_pred, valid_labels).to(device)    
            loss.to(device)
            
            print('loss.item of the valid = ', t, loss.item())  
            valid_loss = valid_loss + loss.item() #* valid_data.size(0)
            
        loss_valid_plt[t]=valid_loss / len(validationloader)   
        
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        
        
        k=0
        for p in range(node_costs.shape[0]):
            for q in range(p+1,node_costs.shape[0]):
                nodeSub[t][k]=node_costs[p][q]
                k=k+1
        k=0
        for p in range(edge_costs.shape[0]):
            for q in range(p+1,edge_costs.shape[0]):
                edgeSub[t][k]=edge_costs[p][q]
                k=k+1
        
            
        print(f'Iteration {t+1} \t\t Training Loss: {\
        train_loss / len(trainloader)} \t\t Validation Loss: {\
        valid_loss / len(validationloader)}')
        if min_valid_loss > valid_loss:
            print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f})')
            min_valid_loss = valid_loss
            iter_min_valid_loss = t
    print('iter and min_valid_loss = ',iter_min_valid_loss, min_valid_loss)
    nodeInsDel_min = InsDel[iter_min_valid_loss][0]
    edgeInsDel_min = InsDel[iter_min_valid_loss][1]
    nodeSub_min = nodeSub[iter_min_valid_loss]
    edgeSub_min = edgeSub[iter_min_valid_loss]
    print(' Min cost for nodeInsDel = ', nodeInsDel_min)
    print(' Min cost for edgeInsDel = ', edgeInsDel_min)
    print(' Min cost for nodeSub = ', nodeSub_min)
    print(' Min cost for edgeSub = ', edgeSub_min)
    torch.save(nodeInsDel_min, 'nodeInsDel_min', pickle_module=pkl) 
    torch.save(edgeInsDel_min, 'edgeInsDel_min', pickle_module=pkl) 
    torch.save(nodeSub_min, 'nodeSub_min', pickle_module=pkl) 
    torch.save(edgeSub_min, 'edgeSub_min', pickle_module=pkl)
    return InsDel,nodeSub,edgeSub,loss_plt,loss_valid_plt,loss_train_plt



In [ ]:
nb_iter=100
InsDel, nodeSub,edgeSub,loss_plt,loss_valid_plt,loss_train_plt=classification(model,data,yt,nb_iter)
plt.figure(0)
plt.plot(InsDel[0:nb_iter,0],label="node")
plt.plot(InsDel[0:nb_iter,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:nb_iter,k])
plt.title('Node Substitutions costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:nb_iter,k])
plt.title('Edge Substitutions costs')
plt.figure(3)
plt.plot(loss_plt)
plt.title('Evolution of the train loss (loss_plt)')

plt.figure(4)
plt.plot(loss_valid_plt)
plt.title('Evolution of the valid loss')
'''
plt.figure(5)
plt.plot(loss_train_plt)
plt.title('Evolution of the loss_train_plt')
'''
plt.show()
plt.close()

loss.item of the train =  0 0.3097071647644043
loss.item of the train =  0 0.31613248586654663
loss.item of the train =  0 0.308906227350235
loss.item of the train =  0 0.31278249621391296
loss.item of the train =  0 0.31432560086250305
ged= tensor([ 0.0168,  0.2172,  0.2269,  0.1500,  0.0104,  0.1547,  0.2311,  0.1206,
         0.0000,  0.0000,  0.1567, -0.1671, -0.5288, -0.9551, -0.9551, -0.1480,
         0.2269,  0.2344,  0.1686,  0.0200,  0.1708,  0.2407,  0.1448,  0.0168,
         0.0168,  0.1753, -0.1833, -0.5459, -0.9480, -0.9480, -0.1605,  0.0184,
         0.1471,  0.1699,  0.1439,  0.0142,  0.1905,  0.1641,  0.1641,  0.1488,
        -0.1491, -0.3409, -0.6603, -0.6603, -0.2110,  0.1406,  0.1672,  0.1245,
         0.0184,  0.1911,  0.1654,  0.1654,  0.1422, -0.1426, -0.3355, -0.6721,
        -0.6721, -0.2171,  0.1434,  0.0132,  0.1702,  0.1489,  0.1334,  0.1334,
         0.0101, -0.0170, -0.3603, -0.7564, -0.7564, -0.1766,  0.1551,  0.2350,
         0.1298,  0.0104,  0.0104,  0.

loss.item of the valid =  3 0.03640921786427498
loss.item of the valid =  3 0.7240738272666931
Iteration 4 		 Training Loss: 0.06118408441543579 		 Validation Loss: 0.2758499579504132
Validation Loss Decreased(1.115440--->1.103400)
loss.item of the train =  4 0.3072173595428467
loss.item of the train =  4 0.30804747343063354
loss.item of the train =  4 0.30842941999435425
loss.item of the train =  4 0.30849331617355347
loss.item of the train =  4 0.30829426646232605
loss.item of the valid =  4 0.2532113492488861
loss.item of the valid =  4 0.0833556205034256
loss.item of the valid =  4 0.06887713074684143
loss.item of the valid =  4 0.7063435912132263
Iteration 5 		 Training Loss: 0.06165885329246521 		 Validation Loss: 0.27794692292809486
loss.item of the train =  5 0.307842880487442
loss.item of the train =  5 0.3071134090423584
loss.item of the train =  5 0.3060375154018402
loss.item of the train =  5 0.30447861552238464
loss.item of the train =  5 0.3023642599582672
loss.item of th

loss.item of the train =  19 0.3042498528957367
loss.item of the train =  19 0.30344799160957336
loss.item of the train =  19 0.30432838201522827
loss.item of the train =  19 0.3046741187572479
loss.item of the train =  19 0.3045992851257324
loss.item of the valid =  19 0.25428563356399536
loss.item of the valid =  19 0.08179108798503876
loss.item of the valid =  19 0.04483089968562126
loss.item of the valid =  19 0.7169824242591858
Iteration 20 		 Training Loss: 0.06091985702514648 		 Validation Loss: 0.2744725113734603
loss.item of the train =  20 0.30413657426834106
loss.item of the train =  20 0.3032567799091339
loss.item of the train =  20 0.30262720584869385
loss.item of the train =  20 0.3058396577835083
loss.item of the train =  20 0.3030627369880676
loss.item of the valid =  20 0.2535097599029541
loss.item of the valid =  20 0.0803881585597992
loss.item of the valid =  20 0.0466737300157547
loss.item of the valid =  20 0.7147407531738281
Iteration 21 		 Training Loss: 0.060612

loss.item of the valid =  34 0.08042748272418976
loss.item of the valid =  34 0.038596805185079575
loss.item of the valid =  34 0.7193196415901184
Iteration 35 		 Training Loss: 0.060706567764282224 		 Validation Loss: 0.27303098421543837
loss.item of the train =  35 0.30373167991638184
loss.item of the train =  35 0.3037812113761902
loss.item of the train =  35 0.3036852777004242
loss.item of the train =  35 0.30345022678375244
loss.item of the train =  35 0.30323126912117004
loss.item of the valid =  35 0.25506269931793213
loss.item of the valid =  35 0.0826963484287262
loss.item of the valid =  35 0.03648655489087105
loss.item of the valid =  35 0.7220323085784912
Iteration 36 		 Training Loss: 0.06064625382423401 		 Validation Loss: 0.27406947780400515
loss.item of the train =  36 0.30455607175827026
loss.item of the train =  36 0.3057042956352234
loss.item of the train =  36 0.3063453137874603
loss.item of the train =  36 0.3066280484199524
loss.item of the train =  36 0.306624382

loss.item of the train =  50 0.30641722679138184
loss.item of the train =  50 0.3063296973705292
loss.item of the train =  50 0.30597934126853943
loss.item of the train =  50 0.307116836309433
loss.item of the valid =  50 0.2493792176246643
loss.item of the valid =  50 0.08184631913900375
loss.item of the valid =  50 0.06284280866384506
loss.item of the valid =  50 0.7034229040145874
Iteration 51 		 Training Loss: 0.061423367261886595 		 Validation Loss: 0.27437281236052513
loss.item of the train =  51 0.3075235188007355
loss.item of the train =  51 0.30705076456069946
loss.item of the train =  51 0.3059374690055847
loss.item of the train =  51 0.3059825897216797
loss.item of the train =  51 0.306126207113266
loss.item of the valid =  51 0.25440725684165955
loss.item of the valid =  51 0.08520917594432831
loss.item of the valid =  51 0.03876996785402298
loss.item of the valid =  51 0.7180854678153992
Iteration 52 		 Training Loss: 0.0612252414226532 		 Validation Loss: 0.27411796711385

loss.item of the valid =  65 0.7220213413238525
Iteration 66 		 Training Loss: 0.06142842769622803 		 Validation Loss: 0.27462209574878216
loss.item of the train =  66 0.3064541518688202
loss.item of the train =  66 0.3055223226547241
loss.item of the train =  66 0.30859342217445374
loss.item of the train =  66 0.3086759150028229
loss.item of the train =  66 0.3056539297103882
loss.item of the valid =  66 0.2534237504005432
loss.item of the valid =  66 0.08724592626094818
loss.item of the valid =  66 0.044152796268463135
loss.item of the valid =  66 0.713409423828125
Iteration 67 		 Training Loss: 0.06113078594207764 		 Validation Loss: 0.2745579741895199
loss.item of the train =  67 0.30628713965415955
loss.item of the train =  67 0.30728888511657715
loss.item of the train =  67 0.308149516582489
loss.item of the train =  67 0.30866482853889465
loss.item of the train =  67 0.30892282724380493
loss.item of the valid =  67 0.2487196922302246
loss.item of the valid =  67 0.08615608513355

loss.item of the train =  81 0.30619606375694275
loss.item of the valid =  81 0.2522604465484619
loss.item of the valid =  81 0.08839605748653412
loss.item of the valid =  81 0.047046761959791183
loss.item of the valid =  81 0.7100505232810974
Iteration 82 		 Training Loss: 0.06123921275138855 		 Validation Loss: 0.27443844731897116
loss.item of the train =  82 0.30618804693222046
loss.item of the train =  82 0.305870920419693
loss.item of the train =  82 0.3052689731121063
loss.item of the train =  82 0.305349200963974


In [ ]:
A=torch.tensor(nx.to_scipy_sparse_matrix(Gs[0],dtype=int,weight='bond_type').todense(),dtype=torch.int) 
print(A)

In [ ]:
plt.plot(loss_plt, label='train loss')
plt.plot(loss_valid_plt, label='valid loss')
plt.title('Train and valid losses')
plt.legend()
plt.show()

In [ ]:
plt.figure(0)
plt.plot(InsDel[0:500,0],label="node")
plt.plot(InsDel[0:500,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:500,k])
plt.title('node Substitution costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:500,k])
plt.title('edge Substitution costs')
plt.show()
plt.close()